In [ ]:

# coding: utf-8

# In[1]:

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
try:
    sess = tf.Session(config=config)
except:
    sess = tf.Session(config=config)
    


# In[2]:

import pickle
transform_list = '/notebook/textData/rank_model/order_data_correct.pickle'

file_data = pickle.load(open(transform_list,'r'))


# In[3]:

len(file_data)


# In[4]:

prod_seq = []
name_seq = []
uuid_seq = []
max_len = 0
max_seq = []
max_name = []
uuid_max = []
data_seq = []
name_dict = {}
not_see_count = 0
count = 0
for index in range(len(file_data)):
    user_order = file_data[index]
    for index_prod in range(len(user_order)):
        count += 1
        one_order = user_order[index_prod]
        id_prod = []
        name_prod = []
        pre_order = ""
        for index_order in range(len(one_order)):
            curr_order = one_order[index_order]
            curr_id = curr_order.split("|")[-2] # get the id
            if curr_id ==pre_order:
                continue
            else:
                id_prod.append(curr_id)
                name_prod.append(curr_order.split("|")[-3])  # get the name
                pre_order = curr_id
            
            if curr_id not in name_dict:
                name_dict[curr_id] = curr_order.split("|")[-3]  # get the sku id 
        # get the not see count
        order_get = id_prod[-1]
        if order_get not in id_prod[:-1]:
            not_see_count += 1
        
            
        if len(id_prod)>=2:
            
            if len(id_prod)>max_len:
                
                max_seq.append(id_prod)
                max_name.append(name_prod)
                max_len = max(len(id_prod), max_len)
                uuid_max.append(curr_order.split("|")[0])
                print len(id_prod)

            prod_seq.append(id_prod)
            name_seq.append(name_prod)
            uuid_seq.append(curr_order.split("|")[0])
            
        
            
        


# In[ ]:




# In[5]:

curr_order


# In[6]:

len(uuid_seq)


# In[7]:

len(prod_seq)


# In[8]:

prod_seq[240]


# In[ ]:




# In[9]:

len(prod_seq)


# In[10]:

# Build dictionary of words
def build_dictionary(sentences, vocabulary_size):
    # Turn sentences (list of strings) into lists of words
    #split_sentences = [s.split() for s in sentences]
    words = [x for sublist in sentences for x in sublist]
    
    # Initialize list of [word, word_count] for each word, starting with unknown
    count = []
    
    # Now add most frequent words, limited to the N-most frequent (N=vocabulary size)
    count.extend(collections.Counter(words).most_common(vocabulary_size-1))
    
    # Now create the dictionary
    word_dict = {}
    # For each word, that we want in the dictionary, add it, then make it
    # the value of the prior dictionary length
    for word, word_count in count:
        word_dict[word] = len(word_dict)
    
    return(word_dict)


# In[ ]:




# In[11]:

sentences = prod_seq


# In[12]:


len(sentences)


# In[ ]:




# In[13]:

import collections

vocabulary_size = min(50000, len(name_dict) )

word_dictionary = build_dictionary(prod_seq, vocabulary_size)
word_dictionary_rev = dict(zip(word_dictionary.values(), word_dictionary.keys()))


# In[19]:

word_dictionary_rev


# In[21]:

len(name_dict)


# In[15]:

len(word_dictionary_rev)


# In[22]:

list_product = []
for key in word_dictionary_rev.keys():
    value = word_dictionary_rev[key]
    if value in name_dict:
        list_product.append(name_dict[value])
    else:
        list_product.append("unk")
    


# In[24]:

len(list_product)


# In[25]:

# Turn text data into lists of integers from dictionary
def text_to_numbers(sentences, word_dict):
    # Initialize the returned data
    data = []
    for sentence in sentences:
        sentence_data = []
        # For each word, either use selected index or rare word index
        for word in sentence:
            if word in word_dict:
                word_ix = word_dict[word]
            else:
                word_ix = 0
            sentence_data.append(word_ix)
        data.append(sentence_data)
    return(data)


# In[26]:

prod_seq[301]


# In[36]:

text_data = text_to_numbers(prod_seq, word_dictionary)


# In[37]:

len(word_dictionary)


# In[38]:

valid_words = prod_seq[30]
valid_examples = [word_dictionary[x] for x in valid_words]


# In[39]:

len(word_dictionary)


# In[40]:

valid_examples[1]


# In[41]:

len(word_dictionary)


# In[42]:

name_dict[word_dictionary_rev[valid_examples[0]]]


# In[33]:

# Get validation word keys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import string
import requests
import collections
import io
import tarfile
from nltk.corpus import stopwords
from six.moves import urllib

import urllib

# Start a graph session


# Declare model parameters
batch_size = 50
embedding_size = 200
generations = 10000
print_loss_every = 500


num_sampled = int(batch_size/2)    # Number of negative examples to sample.
window_size = 4       # How many words to consider left and right.

# Declare stop words
stops = stopwords.words('english')

# We pick five test words. We are expecting synonyms to appear
print_valid_every = 2000

# Generate data randomly (N words behind, target, N words ahead)
def generate_batch_data(sentences, batch_size, window_size, method='skip_gram'):
    # Fill up data batch
    batch_data = []
    label_data = []
    while len(batch_data) < batch_size:
        # select random sentence to start
        rand_sentence = np.random.choice(sentences)
        # Generate consecutive windows to look at
        window_sequences = [rand_sentence[max((ix-window_size),0):(ix+window_size+1)] for ix, x in enumerate(rand_sentence)]
        # Denote which element of each window is the center word of interest
        label_indices = [ix if ix<window_size else window_size for ix,x in enumerate(window_sequences)]
        
        # Pull out center word of interest for each window and create a tuple for each window
        if method=='skip_gram':
            batch_and_labels = [(x[y], x[:y] + x[(y+1):]) for x,y in zip(window_sequences, label_indices)]
            # Make it in to a big list of tuples (target word, surrounding word)
            tuple_data = [(x, y_) for x,y in batch_and_labels for y_ in y]
        elif method=='cbow':
            batch_and_labels = [(x[:y] + x[(y+1):], x[y]) for x,y in zip(window_sequences, label_indices)]
            # Make it in to a big list of tuples (target word, surrounding word)
            tuple_data = [(x_, y) for x,y in batch_and_labels for x_ in x]
        else:
            raise ValueError('Method {} not implemented yet.'.format(method))
            
        # extract batch and labels
        batch, labels = [list(x) for x in zip(*tuple_data)]
        batch_data.extend(batch[:batch_size])
        label_data.extend(labels[:batch_size])
    # Trim batch and label at the end
    batch_data = batch_data[:batch_size]
    label_data = label_data[:batch_size]
    
    # Convert to numpy array
    batch_data = np.array(batch_data)
    label_data = np.transpose(np.array([label_data]).astype(int))
    
    return(batch_data, label_data)
    

# Define Embeddings:
embeddings = tf.Variable(tf.random_uniform([len(word_dictionary), embedding_size], -1.0, 1.0), name = "product_vec")

# NCE loss parameters
nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                               stddev=1.0 / np.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Create data/target placeholders
x_inputs = tf.placeholder(tf.int32, shape=[batch_size])
y_target = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# Lookup the word embedding:
embed = tf.nn.embedding_lookup(embeddings, x_inputs)

# Get loss from prediction
loss_get = tf.nn.nce_loss(weights=nce_weights,            # Tensor of shape(50000, 128)
                          biases=nce_biases,              # vector of zeros; len(128)
                          labels=y_target,            # labels == context words enums
                          inputs=embed,                   # Tensor of shape(128, 128)
                          num_sampled=num_sampled,        # 64: randomly chosen negative (rare) words
                          num_classes=vocabulary_size)   # 50000: by construction
loss = tf.reduce_mean(loss_get)


# Create optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

tf.summary.scalar("loss", loss)

merged_summary_op = tf.summary.merge_all()


# Cosine similarity between words
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)


# In[43]:


#Add variable initializer.
init = tf.global_variables_initializer()
sess.run(init)

LOG_DIR = '/tmp/testing/product2vce'
#tensorboard --logdir=/tmp/testing/example_2_2
summary_writer = tf.summary.FileWriter(LOG_DIR, graph=tf.get_default_graph())

# Run the skip gram model.
loss_vec = []
loss_x_vec = []
for i in range(generations):
    batch_inputs, batch_labels = generate_batch_data(text_data, batch_size, window_size)
    feed_dict = {x_inputs : batch_inputs, y_target : batch_labels}

    # Run the train step
    _, summary = sess.run([optimizer, merged_summary_op], feed_dict=feed_dict)
    
    summary_writer.add_summary(summary, i)

    # Return the loss
    if (i+1) % print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        loss_vec.append(loss_val)
        loss_x_vec.append(i+1)
        print("Loss at step {} : {}".format(i+1, loss_val))
      
    # Validation: Print some random words and top 5 related words
    if (i+1) % print_valid_every == 0:
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = name_dict[word_dictionary_rev[valid_examples[0]]] #word_dictionary_rev[valid_examples[j]]
            top_k = 5 # number of nearest neighbors
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            #name_dict[word_dictionary_rev[valid_examples[0]]]
            log_str = "Nearest to {}::".format(valid_word)
            for k in range(top_k):
                close_word = name_dict[word_dictionary_rev[valid_examples[k]]] #word_dictionary_rev[nearest[k]]
                log_str = "%s %s," % (log_str, close_word)
            print(log_str)
            print "#################"
            


# In[ ]:




# In[45]:

import pandas as pd
new_df = pd.DataFrame() #creates a new dataframe that's empty
new_df["word"] = list_product
import csv
new_df.to_csv(os.path.join(LOG_DIR, 'output2.tsv'), sep='\t', quoting=csv.QUOTE_NONE)


# In[48]:

from tensorflow.contrib.tensorboard.plugins import projector
# Create randomly initialized embedding weights which will be trained.

# Format: tensorflow/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = 'product_vec'
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'output2.tsv')

# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(LOG_DIR)

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver()

saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1000)


# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[57]:

normalized_embeddings


# In[27]:

normalized_embeddings_get = sess.run(normalized_embeddings)


# In[86]:

normalized_embeddings_get.shape


# In[28]:

import pickle

prod2Vec_save = '/notebook/textData/rank_model/product_vec_update_10_July.pickle'

with open(prod2Vec_save, 'wb') as handle:
    pickle.dump(normalized_embeddings_get, handle, protocol=pickle.HIGHEST_PROTOCOL)


# In[29]:

normalized_embeddings_get.shape


# In[30]:

res = word_dictionary_rev.get(5240)


# In[31]:

res


# In[32]:

len(word_dictionary_rev)


# In[33]:

res


# In[34]:

prod2Vec_mapping_sku = '/notebook/textData/rank_model/sku_index_update_10_July.pickle'

with open(prod2Vec_mapping_sku, 'wb') as handle:
    pickle.dump(word_dictionary_rev, handle, protocol=pickle.HIGHEST_PROTOCOL)


# # predict the next product, which is based on the shoping history

# In[99]:

len(text_data)


# In[100]:

len(word_dictionary_rev.keys())


# In[ ]:




# In[255]:

train_x = []
label = []
for item in text_data:
    train_x.append(item[:-1])
    label.append(item[-1])


# In[ ]:




# In[ ]:




# In[ ]:




# In[270]:

hidden_information = 100
dropout_keep_prob = tf.placeholder(tf.float32, name = "dropout_keep_prob")
seq_max_len = 20 # Sequence max length

x_data = tf.placeholder(tf.int32, [ None, seq_max_len], name = "sequence_prod")
y_data = tf.placeholder(tf.int32, [ None, 1], name = "target")

seqlen = tf.placeholder(tf.int32, [None], name = "sequence_length")



y_target = tf.nn.embedding_lookup(normalized_embeddings_get, y_data)

embedding_output = tf.nn.embedding_lookup(normalized_embeddings_get, x_data)


# In[271]:

# cell = tf.nn.rnn_cell.GRUCell(hidden_size)
# rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)
# tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)
# output, out_state = tf.nn.dynamic_rnn(cell, seq, length, initial_state


# In[343]:

with tf.variable_scope('cell_def_2'):
    cellTwo= tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(hidden_information, forget_bias=0.1,
                                                        activation = tf.nn.relu,
                                                        state_is_tuple=True, reuse=tf.get_variable_scope().reuse)
try:
    with tf.variable_scope("lstm_chainThree"):
        output_rnn,final_states=tf.nn.dynamic_rnn(cellTwo, embedding_output, sequence_length = seqlen, dtype=tf.float32)

        #output_rnn,final_states=tf.nn.dynamic_rnn(cellTwo, embedding_output, sequence_length = seqlen, dtype=tf.float32)
        output = tf.nn.dropout(output_rnn, dropout_keep_prob)

        # Get output of RNN sequence,     # and change back dimension to [batch_size, n_step, n_input]
        output = tf.transpose(output, [1, 0, 2])
        # add attension here and categaorial featuers
        last = tf.gather(output, int(output.get_shape()[0]) - 1)
        # concate the categories features    
    
except:
    with tf.variable_scope("lstm_chainThree", reuse = True):
        output_rnn,final_states=tf.nn.dynamic_rnn(cellTwo, embedding_output,  dtype=tf.float32)
        #output_rnn,final_states=tf.nn.dynamic_rnn(cellTwo, embedding_output, sequence_length = seqlen, dtype=tf.float32)
        output = tf.nn.dropout(output_rnn, dropout_keep_prob)

        # Get output of RNN sequence,     # and change back dimension to [batch_size, n_step, n_input]
        output = tf.transpose(output, [1, 0, 2])
        # add attension here and categaorial featuers
        last = tf.gather(output, int(output.get_shape()[0]) - 1)
        # concate the categories features    


# In[344]:

A = tf.Variable(tf.random_normal(shape=[100,200]))
b = tf.Variable(tf.random_normal(shape=[200]))
model_output = tf.add(tf.matmul(last, A), b)


# In[345]:

# Declare model operations


# In[346]:

# Declare loss function (L2 loss)
loss = tf.reduce_mean(tf.square(y_target - model_output))


# In[347]:

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.005)
optimizer_train = my_opt.minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)


# # training the network

# In[277]:

len(batch_seq)


# In[280]:

np.array(batch_seq)


# In[289]:

batch_x = np.array([np.array(x) for x in train_x[epoch*batch:(epoch+1)*batch]])


# In[293]:

batch_y = np.array(label[epoch*batch:(epoch+1)*batch])


# In[295]:

batch_y.shape


# In[297]:

seqlen


# In[298]:

batch_seq


# In[299]:

batch = 50
num_epoch = len(text_data)/batch -1
epoch = 0 
display_step = 3
while epoch<num_epoch:
    batch_x = np.array([np.array(x) for x in train_x[epoch*batch:(epoch+1)*batch]])
    batch_seq = np.array([len(x) for x in batch_x])
    batch_y = np.array(label[epoch*batch:(epoch+1)*batch])

    step = 1
    # Keep training until reach max iterations
        # Run optimization op (backprop)
        
        
    _, c, = sess.run([optimizer_train , loss], feed_dict={x_data: batch_x, y_data: batch_y,
                                   seqlen: batch_seq})
    if step % display_step == 0:
        print "the loss is %f" %c


    step += 1
    print("Optimization Finished!")


# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[234]:

batch_x = text_data[epoch*batch:(epoch+1)*batch]


# In[236]:

len(batch_x)


# In[ ]:




# In[231]:

# train_x = []


# # padding to zero

# In[301]:

batch_x = train_x[epoch*batch:(epoch+1)*batch]


# In[302]:

train_x_array = np.zeros((len(text_data), cut_len))


# In[ ]:




# In[303]:

train_x_array.shape


# In[318]:

len_get = []
count_25 = 0
count_24 = 0
train_x_get = []
label_get = []
cut_len = 20

train_x_array = np.zeros((len(text_data), cut_len))
label_y_array = np.zeros((len(text_data)))
for index in range(len(text_data)):
    item = text_data[index]
    len_get.append(len(item))
    label_mask = item[-1]

    if len(item)>cut_len:
        new_item = item[:cut_len]
        train_x_get.append(item[:cut_len])
        label_get.append(item[cut_len])
        count_25 += 1
    else:
        label_get.append(item[-1])
        new_item = item[:-1]
        for i in range(cut_len):
            new_item.append(0)
            if len(new_item) == cut_len:
                train_x_get.append(new_item)
                break
        count_24 += 1
    train_x_array[index, :] = new_item
    label_y_array[index] = label_mask
        #print new_item


# In[319]:

train_x_array.shape


# In[320]:

label_y_array.shape


# In[324]:

batch_x = train_x_array[epoch*batch:(epoch+1)*batch]


# In[326]:

batch_x.shape


# In[330]:

batch_x.shape


# In[331]:

y_data


# In[360]:

aa = np.reshape(batch_y, (50, 1))


# In[355]:




# In[356]:

import random 


# In[357]:

num_epoch = len(text_data)/batch -1


# In[361]:

label_y_array


# In[ ]:

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)
batch = 50
num_epoch = 200#len(text_data)/batch -1
epoch = 0 
display_step = 10
while epoch<num_epoch:
    epoch = random.randrange(1, num_epoch, 1)
    try:
        batch_x = train_x_array[epoch*batch:(epoch+1)*batch]
        batch_y = np.reshape(label_y_array[epoch*batch:(epoch+1)*batch], (50, 1))
        step = 1
        # Keep training until reach max iterations
            # Run optimization op (backprop)
        _, c, = sess.run([optimizer_train , loss], feed_dict={x_data: batch_x, y_data: batch_y, dropout_keep_prob:0.9})
                                       #seqlen: batch_seq})
        print c
        step += 1
    except:
        continue
    #print("Optimization Finished!")


# In[314]:

batch = 100
num_epoch = len(train_x)/batch -1
epoch = 0 
# while epoch<10:
batch_x = train_x_array[epoch*batch:(epoch+1)*batch]
batch_y = label_y_array[epoch*batch:(epoch+1)*batch]





# In[316]:

batch_x.shape


# In[125]:

num_epoch


# In[ ]:




# In[ ]:

# Training loop
loss_vec = []
for i in range(100):
    rand_index = np.random.choice(len(x_vals), size= batch_size)
    rand_x = x_vals[rand_index]
    rand_y = np.reshape(y_vals[rand_index],(batch_size,1))
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
    if (i+1)%25==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + ' b = ' + str(sess.run(b)))
        print('Loss = ' + str(temp_loss))


# In[114]:

y_target


# In[115]:

hidden_information


# In[ ]:




# In[ ]:

# Neural net parameters

learning_rate = 1e-3
embedding_size = hidden_information
n_labels = 200 #len(np.unique(Y))
n_hidden = 300
n_class = 200
b = tf.Variable(tf.random_normal(shape=[n_class]))


# Neural network weights
try:
    with tf.variable_scope("weight__get_4"):
        h = tf.get_variable(name='h2', shape=[embedding_size, n_hidden],
                            initializer=tf.contrib.layers.xavier_initializer())
        W_out = tf.get_variable(name='out_w', shape=[n_hidden, n_class],
                                initializer=tf.contrib.layers.xavier_initializer())
except:
    with tf.variable_scope("weight__get_4", reuse = True):
        h = tf.get_variable(name='h2', shape=[embedding_size, n_hidden],
                            initializer=tf.contrib.layers.xavier_initializer())
        W_out = tf.get_variable(name='out_w', shape=[n_hidden, n_class],
                                initializer=tf.contrib.layers.xavier_initializer())
    


# In[ ]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import learn
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn import cross_validation
import numpy as np

boston = learn.datasets.load_dataset('boston')
x, y = boston.data, boston.target
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(
x, y, test_size=0.2, random_state=42)

total_len = X_train.shape[0]

# Parameters
learning_rate = 0.001
training_epochs = 500
batch_size = 10
display_step = 1
dropout_rate = 0.9
# Network Parameters
n_hidden_1 = 32 # 1st layer number of features
n_hidden_2 = 200 # 2nd layer number of features
n_hidden_3 = 200
n_hidden_4 = 256
n_input = X_train.shape[1]
n_classes = 1

# tf Graph input
x = tf.placeholder("float", [None, 13])
y = tf.placeholder("float", [None])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)

    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, 0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, 0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], 0, 0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes], 0, 0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], 0, 0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, 0.1)),
    'b3': tf.Variable(tf.random_normal([n_hidden_3], 0, 0.1)),
    'b4': tf.Variable(tf.random_normal([n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_classes], 0, 0.1))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.square(pred-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(total_len/batch_size)
        # Loop over all batches
        for i in range(total_batch-1):
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = Y_train[i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        # sample prediction
        label_value = batch_y
        estimate = p
        err = label_value-estimate
        print ("num batch:", total_batch)

        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=",                 "{:.9f}".format(avg_cost))
            print ("[*]----------------------------")
            for i in xrange(3):
                print ("label value:", label_value[i],                     "estimated value:", estimate[i])
            print ("[*]============================")

    print ("Optimization Finished!")

#     # Test model
#     correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#     # Calculate accuracy
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#     print ("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))


# In[ ]:




# In[ ]:




# In[108]:

x_data


# In[91]:

last


# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# # For the parameters

# In[ ]:




# In[ ]:




# In[64]:

seq_max_len = 20
n_hidden = 64


# In[69]:

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 1])
y = tf.placeholder("float", [None, None])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])


# In[70]:

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, 2]))
}
biases = {
    'out': tf.Variable(tf.random_normal([2]))
}


# In[72]:

hidden_information = 100
dropout_keep_prob = tf.placeholder(tf.float32, name = "dropout_keep_prob")

x_data = tf.placeholder(tf.int32, [ None, 25], name = "search_term")

embedding_output = tf.nn.embedding_lookup(normalized_embeddings_get, x_data)


with tf.variable_scope('cell_def_2'):
    cellTwo= tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(hidden_information, forget_bias=0.1,
                                                        activation = tf.nn.relu,
                                                        state_is_tuple=True, reuse=tf.get_variable_scope().reuse)


try:
    with tf.variable_scope("lstm_chainThree"):
        output_rnn,final_states=tf.nn.dynamic_rnn(cellTwo, embedding_output, dtype=tf.float32)
        output = tf.nn.dropout(output_rnn, dropout_keep_prob)

        # Get output of RNN sequence,     # and change back dimension to [batch_size, n_step, n_input]
        output = tf.transpose(output, [1, 0, 2])
        # add attension here and categaorial featuers
        last = tf.gather(output, int(output.get_shape()[0]) - 1)
        # concate the categories features    
    
except:
    with tf.variable_scope("lstm_chainThree", reuse = True):
        output_rnn,final_states=tf.nn.dynamic_rnn(cellTwo, embedding_output, dtype=tf.float32)
        output = tf.nn.dropout(output_rnn, dropout_keep_prob)

        # Get output of RNN sequence,     # and change back dimension to [batch_size, n_step, n_input]
        output = tf.transpose(output, [1, 0, 2])
        # add attension here and categaorial featuers
        last = tf.gather(output, int(output.get_shape()[0]) - 1)
        # concate the categories features    


# In[73]:

last


# In[75]:


# Neural net parameters

learning_rate = 1e-3
embedding_size = hidden_information
#n_labels = len(np.unique(Y))
n_hidden = 60
n_class = 2
b = tf.Variable(tf.random_normal(shape=[2]))


# Neural network weights
try:
    with tf.variable_scope("weight__get_4"):
        h = tf.get_variable(name='h2', shape=[embedding_size, n_hidden],
                            initializer=tf.contrib.layers.xavier_initializer())
        W_out = tf.get_variable(name='out_w', shape=[n_hidden, n_class],
                                initializer=tf.contrib.layers.xavier_initializer())
except:
    with tf.variable_scope("weight__get_4", reuse = True):
        h = tf.get_variable(name='h2', shape=[embedding_size, n_hidden],
                            initializer=tf.contrib.layers.xavier_initializer())
        W_out = tf.get_variable(name='out_w', shape=[n_hidden, n_class],
                                initializer=tf.contrib.layers.xavier_initializer())
    


# In[ ]:

layer_1 = tf.matmul(combine_tensor,h)

layer_activate = tf.nn.relu(layer_1)


# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# # image relevance analysis

# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# # relevance analysis 

# In[35]:

import pickle
transform_list = '/notebook/textData/rank_model/nearest.pickle'

nearest_data_prod2vec = pickle.load(open(transform_list,'r'))


# In[36]:

import os

directory = '/notebook/textData/rank_model/image_get/imag_get'
data_get = []
import glob
dict_get = {}
for filename in glob.glob(os.path.join(directory, '*.jpg')):
    dict_get[filename.split("/")[-1][:-4]] = filename


# In[37]:

len(nearest_data)


# In[ ]:

import random 
data_index = random.sample(range(len(nearest_data) -1), 30)


# In[ ]:

nearest_data[data_index[1]]


# In[ ]:

data_index


# In[ ]:




# In[ ]:

list_data


# # get the name and price and sku weights

# In[ ]:

data_path = '/notebook/textData/rank_model/disney_products.json'

import json

data = []
sku_name = {}
sku_price = {}
with open(data_path) as f:
    for line in f:
        item = json.loads(line)
        data.append(item)
        if "name" in item.keys():
            sku_name[item["sku"]] = item["name"]
        elif "<PRODUCTNAME" in item.keys():
            sku_name[item["sku"]] = item["<PRODUCTNAME"]
        else:
            print (item)
        if "final_price" in item.keys():
            sku_price[item["sku"]] = item["final_price"]


# In[ ]:

len(sku_name)


# In[ ]:

import pickle
import pandas as pd
transform_list = '/notebook/textData/rank_model/sku_wight.csv'

sku_weight = pd.read_csv(transform_list)
sku_weights_valid = sku_weight[['sku','weight']]


# In[ ]:

item["price"]


# In[ ]:

item["sku_fallback"]


# In[ ]:

sku_weights_valid.columns


# In[ ]:

sku_weights_dict = dict(zip(sku_weights_valid['sku'], sku_weights_valid["weight"]))


# In[ ]:




# In[ ]:




# In[ ]:

len(dict_get)


# In[ ]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
print ('the next view sequence')

def plot_list(sku_sequence, sku_name, sku_price,sku_weights_valid):
    for flat_list in sku_sequence:
        print ("#########")
        print (flat_list)
        for key in flat_list:
            try:                
                print ("the product name is")
                print (sku_name[key])
                print ("the price is %f" % sku_price[key])
#                 if key in sku_weights_dict.keys():
#                     sku_weights_valid[key]
#                     print ("the weight is %f" % sku_weights_valid[key])
                print "show the image"

                plt.figure()
                img=mpimg.imread(dict_get[key])
                imgplot = plt.imshow(img)
                plt.show()
            except:
                continue


# # product2vec nearly neighbors

# In[ ]:

import pickle
transform_list = '/notebook/textData/rank_model/nearest_update_one.pickle'

nearest_data_image = pickle.load(open(transform_list,'r'))


# In[ ]:

nearest_data_image


# In[ ]:

len(nearest_data_image)


# In[ ]:

data_index = random.sample(range(1000), 30)


# In[ ]:

def get_NN(data_index, nearest_data_prod2vec):
    list_data = []
    for item_query in data_index: 
        list_one = nearest_data_prod2vec[item_query]

        flat_list = [list_one[0]]
        for item in list_one[1]:
            flat_list.append(item)
        list_data.append(flat_list)
    return list_data


# In[ ]:

list_data = get_NN(data_index, nearest_data_prod2vec)
list_image = get_NN(data_index, nearest_data_image)


# In[ ]:

nearest_data_prod2vec


# In[ ]:

prod2vec = {}
for item in nearest_data_prod2vec:
    prod2vec[item[0]] = item[1]
    
    


# In[ ]:

image2vec = {}

for item in nearest_data_image:
    image2vec[item[0]] = item[1]
    


# In[ ]:

result_get = set(image2vec).intersection(prod2vec)


# In[ ]:

list_data = []
list_data_image = []
for item in result_get:
    list_data.append([item] + prod2vec[item])
    list_data_image.append([item, item] + image2vec[item])
    


# In[ ]:

list_data_image


# In[ ]:

list_data


# # comparing the index part

# In[ ]:

plot_list(list_data, sku_name, sku_price,sku_weights_valid)


# In[ ]:

plot_list(list_data_image, sku_name, sku_price,sku_weights_valid)


# In[ ]:

#close_word = name_dict[word_dictionary_rev[valid_examples[k]]] #word_dictionary_rev[nearest[k]]


# # show the images

# In[ ]:

# Step 6: Visualize the embeddings.
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

plot_only = 20
valid_examples = range(plot_only)

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=35, n_components=2, init='pca', n_iter=5000)

  low_dim_embs = tsne.fit_transform(normalized_embeddings_get[:plot_only, :])
  labels = [unicode(name_dict[word_dictionary_rev[valid_examples[i]]], "latin-1") for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)
except ImportError:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')


# In[ ]:


valid_examples


# In[ ]:

close_word = name_dict[word_dictionary_rev[valid_examples[i]]] #word_dictionary_rev[nearest[k]]


# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# coding: utf-8

# In[9]:

import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.__version__

PATH = "/notebook/textData/rank_model/"

LOG_DIR = PATH + 'embedding-logs_13'


# In[10]:

import pickle


# In[11]:


prod2Vec_save = '/notebook/textData/rank_model/product_vec_update_10_July.pickle'
prod2Vec_mapping_sku = '/notebook/textData/rank_model/sku_index_update_10_July.pickle'


index_sku = pickle.load(open(prod2Vec_mapping_sku,'r'))


# In[12]:

len(index_sku)


# In[13]:

import os

directory = '/notebook/textData/rank_model/image_get/imag_get'
data_get = []
import glob
dict_get = {}
for filename in glob.glob(os.path.join(directory, '*.jpg')):
    dict_get[filename.split("/")[-1][:-4]] = filename


# In[14]:

id_get = index_sku.values()


# In[15]:

number_items = len(id_get)


# In[16]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img_data=[]
count = 0
for key in id_get:
    try:
        #plt.figure()
        img=mpimg.imread(dict_get[key])
        input_img_resize=cv2.resize(img,(100, 100))
        img_data.append(input_img_resize)
        
        if count %1000 ==0:
            print (count)

        #imgplot = plt.imshow(input_img_resize)
        #plt.show()
    except:
        continue
    count += 1
    


# In[17]:

img_data = np.array(img_data)
imgplot = plt.imshow(img_data[144])
plt.show()


# In[18]:

import pickle
prod2Vec_save_path = '/notebook/textData/rank_model/product_vec_update_10_July.pickle'
prod2Vec_save = pickle.load(open(prod2Vec_save_path,'r'))


# In[19]:

feature_vectors = np.array(prod2Vec_save).reshape((number_items, 200))


# In[20]:

feature_vectors.shape


# In[ ]:




# In[21]:

data_path = '/notebook/textData/rank_model/disney_products.json'

import json

data = []
sku_name = {}
sku_price = {}
with open(data_path) as f:
    for line in f:
        item = json.loads(line)
        data.append(item)
        if "name" in item.keys():
            sku_name[item["sku"]] = item["name"]
        elif "<PRODUCTNAME" in item.keys():
            sku_name[item["sku"]] = item["<PRODUCTNAME"]
        else:
            print (item)
        if "final_price" in item.keys():
            sku_price[item["sku"]] = item["final_price"]


# In[22]:

name_list = []


# In[ ]:




# In[23]:

for key in id_get:
    name_list.append(sku_name.get(key, "None").encode('ascii', 'ignore').decode('ascii'))


# In[ ]:




# In[24]:

num_of_samples=feature_vectors.shape[0]
num_of_samples_each_class = 10

features = tf.Variable(feature_vectors, name='features')




# In[ ]:




# In[25]:

import sys
reload(sys)
sys.setdefaultencoding('utf8')


# In[26]:



try:
    os.mkdir(LOG_DIR)
except:
    pass
import pandas as pd
new_df = pd.DataFrame() #creates a new dataframe that's empty
new_df["word"] = name_list
import csv
new_df.to_csv(os.path.join(LOG_DIR, 'metadata_6_classes.tsv'), sep='\t', quoting=csv.QUOTE_NONE)


# In[ ]:




# In[27]:

# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data


# In[28]:

sprite = images_to_sprite(img_data)


# In[29]:

cv2.imwrite(os.path.join(LOG_DIR, 'sprite_4_classes.png'), sprite)


# In[30]:

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
try:
    sess = tf.Session(config=config)
except:
    sess = tf.Session(config=config)
    


# In[31]:

saver = tf.train.Saver([features])

sess.run(features.initializer)
saver.save(sess, os.path.join(LOG_DIR, 'images_5_classes.ckpt'))

config = projector.ProjectorConfig()
# One can add multiple embeddings.
embedding = config.embeddings.add()
embedding.tensor_name = features.name
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata_6_classes.tsv')
# Comment out if you don't want sprites
embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite_4_classes.png')
embedding.sprite.single_image_dim.extend([img_data.shape[1], img_data.shape[1]])
# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


# In[32]:

LOG_DIR


# In[ ]:



